<a href="https://colab.research.google.com/github/simecek/dspracticum2020/blob/master/lecture_08/assignment/SimpleGenomicNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

In [1]:
!pip install biopython

     |████████████████████████████████| 2.3MB 9.4MB/s 


In [2]:
import urllib.request
from pathlib import Path
from Bio import SeqIO
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

## Reshaping data from fasta to txt

In [3]:
classes = ['nonpromoters', 'promoters']
sets = ['train', 'valid', 'test']

for c in classes:
    for s in sets:
        urllib.request.urlretrieve(f"https://raw.githubusercontent.com/katarinagresova/benchmarks/main/ecoli_promoters/e_coli_{c}_{s}.fa", f"e_coli_{c}_{s}.fa")

In [4]:
for c in classes:
    for s in sets:
        Path(f"data/{s}/{c}").mkdir(parents=True, exist_ok=True)

In [5]:
for c in classes:
    for s in sets:
        with open(f"e_coli_{c}_{s}.fa", "r") as fr:
            for record in SeqIO.parse(fr, "fasta"):
                id = record.id
                with open(f"data/{s}/{c}/{id}.txt", "w") as fw:
                    fw.writelines([" ".join(str(record.seq))])


## Reading data

In [6]:
batch_size = 64

raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'data/train/',
    batch_size=batch_size,
    class_names=classes)

Found 6791 files belonging to 2 classes.


In [7]:
raw_valid_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'data/valid/',
    batch_size=batch_size,
    class_names=classes)

raw_test_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'data/test/',
    batch_size=batch_size,
    class_names=classes)

Found 750 files belonging to 2 classes.
Found 830 files belonging to 2 classes.


In [8]:
text_batch, label_batch = next(iter(raw_train_ds))
text_batch[0], label_batch[0]

(<tf.Tensor: shape=(), dtype=string, numpy=b'G G G A T C A C C G C C T T C T T G T C G G T G G C G A T A T T A T T G T C T T T G C T G A T C T T T A T T G G T G A A G C C G T C C G C G A C G C A T T T G A T C C T A A T A A G G C G G T G T A G C A T G A C G C A A A C T C T G T T A G C G A T T G A A A A T T T G T C G G T G G G T T T T C G C C A'>,
 <tf.Tensor: shape=(), dtype=int32, numpy=1>)

In [9]:
vectorize_layer = TextVectorization(output_mode='int')


In [10]:
train_text = raw_train_ds.map(lambda x, y: x)
vectorize_layer.adapt(train_text)
vectorize_layer.set_vocabulary(vocab=np.asarray(['a', 'c', 't', 'g', 'n']))

In [11]:
vectorize_layer.get_vocabulary()

['', '[UNK]', 'a', 'c', 't', 'g', 'n']

In [12]:
def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer(text)-2, label

In [13]:
text_batch[0], label_batch[0]

(<tf.Tensor: shape=(), dtype=string, numpy=b'G G G A T C A C C G C C T T C T T G T C G G T G G C G A T A T T A T T G T C T T T G C T G A T C T T T A T T G G T G A A G C C G T C C G C G A C G C A T T T G A T C C T A A T A A G G C G G T G T A G C A T G A C G C A A A C T C T G T T A G C G A T T G A A A A T T T G T C G G T G G G T T T T C G C C A'>,
 <tf.Tensor: shape=(), dtype=int32, numpy=1>)

In [14]:
first_review, first_label = text_batch[0], label_batch[0]
vectorize_text(first_review, first_label)

(<tf.Tensor: shape=(1, 150), dtype=int64, numpy=
 array([[3, 3, 3, 0, 2, 1, 0, 1, 1, 3, 1, 1, 2, 2, 1, 2, 2, 3, 2, 1, 3, 3,
         2, 3, 3, 1, 3, 0, 2, 0, 2, 2, 0, 2, 2, 3, 2, 1, 2, 2, 2, 3, 1, 2,
         3, 0, 2, 1, 2, 2, 2, 0, 2, 2, 3, 3, 2, 3, 0, 0, 3, 1, 1, 3, 2, 1,
         1, 3, 1, 3, 0, 1, 3, 1, 0, 2, 2, 2, 3, 0, 2, 1, 1, 2, 0, 0, 2, 0,
         0, 3, 3, 1, 3, 3, 2, 3, 2, 0, 3, 1, 0, 2, 3, 0, 1, 3, 1, 0, 0, 0,
         1, 2, 1, 2, 3, 2, 2, 0, 3, 1, 3, 0, 2, 2, 3, 0, 0, 0, 0, 2, 2, 2,
         3, 2, 1, 3, 3, 2, 3, 3, 3, 2, 2, 2, 2, 1, 3, 1, 1, 0]])>,
 <tf.Tensor: shape=(), dtype=int32, numpy=1>)

In [15]:
train_ds = raw_train_ds.map(vectorize_text)
valid_ds = raw_valid_ds.map(vectorize_text)
test_ds =  raw_test_ds.map(vectorize_text)

## Model training

In [16]:
# one-hot encoding
onehot_layer = keras.layers.Lambda(lambda x: tf.one_hot(tf.cast(x,'int64'), 4))

In [19]:
onehot_layer([-1,0,1,2,3,4])

<tf.Tensor: shape=(6, 4), dtype=float32, numpy=
array([[0., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 0.]], dtype=float32)>

In [49]:
model_type = "LSTM"

if model_type == "LSTM":
    model = tf.keras.Sequential([
          onehot_layer,
          keras.layers.LSTM(128, return_sequences=True),
          keras.layers.Dropout(0.3),
          keras.layers.LSTM(128, return_sequences=False),
          keras.layers.Dropout(0.3),
          keras.layers.Dense(1)])                     
else:
    model = tf.keras.Sequential([
        onehot_layer,
        keras.layers.Conv1D(32, kernel_size=8, data_format='channels_last', activation='relu'),
        keras.layers.BatchNormalization(),
        keras.layers.MaxPooling1D(),
        keras.layers.Conv1D(16, kernel_size=8, data_format='channels_last', activation='relu'),
        keras.layers.BatchNormalization(),
        keras.layers.MaxPooling1D(),
        keras.layers.Conv1D(4, kernel_size=8, data_format='channels_last', activation='relu'),
        keras.layers.BatchNormalization(),
        keras.layers.MaxPooling1D(),
        keras.layers.Dropout(0.3),
        keras.layers.GlobalAveragePooling1D(),
        keras.layers.Dense(1)])

In [50]:
model.compile(loss=keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=tf.metrics.BinaryAccuracy(threshold=0.0))

In [51]:
epochs = 10
history = model.fit(
    train_ds,
    epochs=epochs,
    validation_data = valid_ds)

Epoch 1/10
107/107 [==============================] - 4s 34ms/step - loss: 0.6962 - binary_accuracy: 0.5353 - val_loss: 0.6951 - val_binary_accuracy: 0.5000
Epoch 2/10
107/107 [==============================] - 3s 32ms/step - loss: 0.6979 - binary_accuracy: 0.5110 - val_loss: 0.6906 - val_binary_accuracy: 0.5493
Epoch 3/10
107/107 [==============================] - 3s 32ms/step - loss: 0.6862 - binary_accuracy: 0.5466 - val_loss: 0.7241 - val_binary_accuracy: 0.5813
Epoch 4/10
107/107 [==============================] - 3s 32ms/step - loss: 0.6271 - binary_accuracy: 0.6672 - val_loss: 0.5246 - val_binary_accuracy: 0.7613
Epoch 5/10
107/107 [==============================] - 3s 33ms/step - loss: 0.5112 - binary_accuracy: 0.7675 - val_loss: 0.5352 - val_binary_accuracy: 0.7467
Epoch 6/10
107/107 [==============================] - 3s 32ms/step - loss: 0.4706 - binary_accuracy: 0.7866 - val_loss: 0.4869 - val_binary_accuracy: 0.7773
Epoch 7/10
107/107 [==============================] - 3s 3

In [ ]:
model.evaluate(test_ds)

13/13 [==============================] - 0s 10ms/step - loss: 0.6834 - binary_accuracy: 0.5570


[0.6833918690681458, 0.5569813251495361]